# 引言

在本次IR实验中，请运用所学的知识，为搜索引擎构建一个简单的索引和检索系统，包括：

1. [构建非压缩索引及检索实现](#构建非压缩索引及检索实现) 在stanford.edu网页语料库上构建未压缩索引，并实现布尔查询检索。
2. [构建压缩索引](#构建压缩索引) 使用可变长度编码在同一个语料库上构建压缩索引，并实现布尔查询。
3. [附加题](#其他压缩方法实现(选做))


## 实验要求及提交说明

1\. 本次实验将于学期结束前提交，具体时间将提前至少2周发布。

2\. 实验项目中包括：

**本实验文档PA1-skeleton.ipynb**

**实验环境安装说明README.md**

**测试语料库文件夹toy-data**

**对完整语料库的查询文件dev_queries**

**完整的语料库压缩文件**

**其他配置文件**

实验推荐安装Anaconda平台（python3版本），具体安装方法请参考网上教程。

2\. 实验需要提交内容包括：

**一份实验报告**（文件名：姓名_学号）

**通过dev_queries文件夹内的查询得到的查询结果文件，命名为dev_out，文件夹内命名规则与dev_queries保持一致**

**实验项目所有内容的压缩文件**（注意：不要任意修改文件名，便于助教进行测试）

其中，实验报告的内容需要包括以下内容：

(1) 需要补充的关键代码及说明；

(2) 所有存在输出结果的结果输出截图（包括测试部分）；

(3) （选做）附加部分的代码说明及分析；

(4) 实验总结。

3\. 请将以上所有提交内容打包到一个压缩文件中，命名为IREXP_姓名_学号，发送至邮箱：irdm_ustc@163.com 。

4\. 本实验参考Stanfod相关课程实验，故以下内容以英文为主。由于本学年第一次新增IR部分实验，如对实验内容有任何问题，欢迎与助教讨论或在QQ群内讨论，助教邮箱：gxq13@mail.ustc.edu.cn 。

## Setup

In [1]:
#Load the tee magic which saves a copy of the cell when executed
%reload_ext autograding_magics

The `submission` folder will contain all the files to be submitted.

In [2]:
import os
try: 
    os.mkdir('submission')
except FileExistsError:
    pass

In [3]:
%%tee submission/imports.py

# You can add additional imports here
import sys
import pickle as pkl
import array
import os
import timeit
import contextlib

import re
import struct

Overwriting submission/imports.py


# Corpus （语料库）

The corpus you will be working with for this assignment contains webpages from the stanford.edu domain. The data for this assignment is available as a .zip file at: http://web.stanford.edu/class/cs276/pa/pa1-data.zip. The following code puts the data folder under the current directory. The total size of the corpus is about 170MB.
（该文件已提前下载在实验文件中）

In [4]:
if os.path.isdir('pa1-data'):
    print('already unziped')
else:

    import urllib.request
    import zipfile

    #'http://web.stanford.edu/class/cs276/pa/pa1-data.zip'
    data_dir = 'pa1-data'
    #urllib.request.urlretrieve(data_url, data_dir+'.zip')
    zip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')
    zip_ref.extractall()
    zip_ref.close()

already unziped


The index will be stored in `output_dir` and `tmp` will contain some temporary files for toy indices.

In [5]:
try: 
    os.mkdir('output_dir')
except FileExistsError:
    pass
try: 
    os.mkdir('tmp')
except FileExistsError:
    pass
try: 
    os.mkdir('toy_output_dir')
except FileExistsError:
    pass

There are 10 sub-directories (named 0-9) under the data directory.

In [6]:
sorted(os.listdir('pa1-data'))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

Each file under the sub-directory is the content of an individual webpage. You should assume that the individual file names are unique within each sub-directory, but not necessarily the case across sub-directories (*i.e.,* the full path of the files are unique).

In [7]:
sorted(os.listdir('pa1-data/0'))[:10]

['3dradiology.stanford.edu_',
 '3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html',
 '3dradiology.stanford.edu_patient_care_case_studies.html',
 '5-sure.stanford.edu_',
 '50years.stanford.edu_',
 'a3cservices.stanford.edu_awards_nominate_',
 'a3cservices.stanford.edu_facilities_',
 'a3cservices.stanford.edu_lead_',
 'aa.stanford.edu_',
 'aa.stanford.edu_about_aviation.php']

All the HTML info has been stripped off from those webpages, so they only contain space-delimited words. You should not re-tokenize the words. Each consecutive span of non-space characters consists of a word token in the corpus.

In [8]:
with open('pa1-data/0/3dradiology.stanford.edu_', 'r') as f:
    print(f.read())

3d radiology lab stanford university school of medicine stanford school of medicine 3d and quantitative imaging in the department of radiology search this site only stanford medical sites ways to give find a person alumni lane library ways to give find a person about us mission to develop and apply innovative techniques for efficient quantitative analysis and display of medical imaging data through interdisciplinary collaboration goals education to train physicians and technologists locally and worldwide in the latest developments in 3d and quantitative imaging research to develop new approaches to the exploration analysis and quantitative assesment of diagnostic images that result in a new and or more cost effective diagnostic approaches and b new techniques for the design and planning and monitoring of therapy patient care to deliver valid clinically relevant visualization and analysis of medical imaging data to the stanford community locations richard m lucas magnetic resonance imag

You will also find a small corpus included with the assignment in the folder `toy-data`. We will use this toy dataset to test our code before running on the complete dataset.

In [9]:
toy_dir = 'toy-data'

# 构建非压缩索引及检索实现

The first part of this assignment is to build an inverted index of this corpus, and implement Boolean conjunctive queries. **In particular, you need to implement the blocked sort-based indexing (BSBI) algorithm described in [Section 4.2](http://nlp.stanford.edu/IR-book/pdf/04const.pdf) of the textbook.** While we quote some fundamentals from the textbook, we strongly suggest that you read Section 4.2 in more detail to fully understand the BSBI algorithm.

实验的第一部分是建立这个语料库的倒排索引，并实现布尔查询（包括实现教材4.2节中描述的BSBI算法）。


> To construct an index, we first make a pass through the collection assembling all term-docID pairs. We then sort the pairs with the term as the dominant key and docID as the secondary key. Finally, we organize the docIDs for each term into a postings list and compute statistics like term and document frequency. For small collections, all this can be done in memory. 

In this assignment, you will be implementing methods for large collections that require the use of secondary storage (*i.e.,* disk).

## IdMap

Again quoting from Section 4.2:

> To make index construction more efficient, we represent terms as termIDs (instead of strings), where each termID is a unique serial number. We can build the mapping from terms to termIDs on the fly while we are processing the collection. Similarly, we also represent documents as docIDs (instead of strings).

Let us first build a helper class `IdMap`, which maps strings to numeric ids (and vice-versa). We will need this to bijectively map term to termID and doc to docID.

Fill in the functions `_get_str` and `_get_id` in the following code. The only interface to this class is provided by `__getitem__` which gets the correct mapping depending on the type of the key. (See [these docs](https://docs.python.org/3.7/reference/datamodel.html#special-method-names) if you are unfamiliar with special functions like `__getitem__` ). Further, to simplify code later on, we'll also add some logic to add a string if it doesn't already exist in the map.   

We will use a dictionary for efficient access from strings to numeric ids, and we will use a list for efficient access (and storage) from numeric ids to strings.

You might also find this [short tutorial](https://www.omkarpathak.in/2018/04/11/python-getitem-and-setitem/) useful to get started with special (a.k.a magic) methods


In [10]:
%%tee submission/id_map.py
class IdMap:
    """Helper class to store a mapping from strings to ids."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        
    def __len__(self):
        """Return number of terms stored in the IdMap"""
        return len(self.id_to_str)
        
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        try:
            string = self.id_to_str[i]
        except IndexError:
            raise
        else:
            return string
        

        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). 
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code
        try:
            id = self.str_to_id[s]
        except KeyError:
            self.id_to_str.append(s)
            self.str_to_id[s] = id = self.__len__() - 1
        return id

        ### End your code
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

Overwriting submission/id_map.py


Make sure it passes the following simple test cases

In [11]:
testIdMap = IdMap()
assert testIdMap['a'] == 0, "Unable to add a new string to the IdMap"
assert testIdMap['bcd'] == 1, "Unable to add a new string to the IdMap"
assert testIdMap['a'] == 0, "Unable to retrieve the id of an existing string"
assert testIdMap[1] == 'bcd', "Unable to retrive the string corresponding to a\
                                given id"
try:
    testIdMap[2]
except IndexError as e:
    assert True, "Doesn't throw an IndexError for out of range numeric ids"
assert len(testIdMap) == 2

Going forward we will expect you to write your own test cases to make sure parts of your program are working as expected.

## Encoding Postings List as bytearrays

In order to write and read lists of postings (docIDs) efficiently from the disk, we store them as bytearrays. We've provided an implementation of `UncompressedPostings` class which contains static encode and decode functions. In the next task you'll be required to implement compressed versions with the same inferface. 

For reference:
1. https://docs.python.org/3/library/array.html
2. https://pymotw.com/3/array/#module-array

In [12]:
class UncompressedPostings:
    
    @staticmethod
    def encode(postings_list):
        """Encodes postings_list into a stream of bytes
        
        Parameters
        ----------
        postings_list: List[int]
            List of docIDs (postings)
            
        Returns
        -------
        bytes
            bytearray representing integers in the postings_list
        """
        return array.array('L', postings_list).tobytes()
        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes postings_list from a stream of bytes
        
        Parameters
        ----------
        encoded_postings_list: bytes
            bytearray representing encoded postings list as output by encode 
            function
            
        Returns
        -------
        List[int]
            Decoded list of docIDs from encoded_postings_list
        """
        
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        return decoded_postings_list.tolist()

To illustrate how it works, run the following cell

In [13]:
x = UncompressedPostings.encode([1,2,3])
print(x)
print(UncompressedPostings.decode(x))

b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00'
[1, 2, 3]


## Inverted Index on Disk

> With main memory insufficient, we need to use an external sorting algorithm, that is, one that uses disk. For acceptable speed, the central requirement of such an algorithm is that it minimize the number of random disk seeks during sorting - sequential disk reads are far faster than seeks. 

In this section we provide a base class `InvertedIndex` which would be subsequently subclassed into `InvertedIndexWriter`, `InvertedIndexIterator` and `InvertedIndexMapper`. While typically `cPickle` is used for serialization in Python, it does not support partial reads and writes, which is why we are rolling out our own custom solution.

In [14]:
class InvertedIndex:
    """A class that implements efficient reads and writes of an inverted index 
    to disk
    
    Attributes
    ----------
    postings_dict: Dictionary mapping: termID->(start_position_in_index_file, 
                                                number_of_postings_in_list,
                                               length_in_bytes_of_postings_list)
        This is a dictionary that maps from termIDs to a 3-tuple of metadata 
        that is helpful in reading and writing the postings in the index file 
        to/from disk. This mapping is supposed to be kept in memory. 
        start_position_in_index_file is the position (in bytes) of the postings 
        list in the index file
        number_of_postings_in_list is the number of postings (docIDs) in the 
        postings list
        length_in_bytes_of_postings_list is the length of the byte 
        encoding of the postings list
    
    terms: List[int]
        A list of termIDs to remember the order in which terms and their 
        postings lists were added to index. 
        
        After Python 3.7 we technically no longer need it because a Python dict 
        is an OrderedDict, but since it is a relatively new feature, we still
        maintain backward compatibility with a list to keep track of order of 
        insertion. 
    """
    def __init__(self, index_name, postings_encoding=None, directory=''):
        """
        Parameters
        ----------
        index_name (str): Name used to store files related to the index
        postings_encoding: A class implementing static methods for encoding and 
            decoding lists of integers. Default is None, which gets replaced
            with UncompressedPostings
        directory (str): Directory where the index files will be stored
        """

        self.index_file_path = os.path.join(directory, index_name+'.index')
        self.metadata_file_path = os.path.join(directory, index_name+'.dict')

        if postings_encoding is None:
            self.postings_encoding = UncompressedPostings
        else:
            self.postings_encoding = postings_encoding
        self.directory = directory

        self.postings_dict = {}
        self.terms = []         #Need to keep track of the order in which the 
                                #terms were inserted. Would be unnecessary 
                                #from Python 3.7 onwards

    def __enter__(self):
        """Opens the index_file and loads metadata upon entering the context"""
        # Open the index file
        self.index_file = open(self.index_file_path, 'rb+')

        # Load the postings dict and terms from the metadata file
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()                       

        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        """Closes the index_file and saves metadata upon exiting the context"""
        # Close the index file
        self.index_file.close()
        
        # Write the postings dict and terms to the metadata file
        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)

Since we are interacting with a file on disk, we provide `__enter__` and `__exit__` functions, which allow using the `with` statement just like you would with file IO in python (Refer to the [official documentation for context managers](https://docs.python.org/3/library/contextlib.html)).

Here is an example of how to use the `InvertedIndexWriter` context manager:

```python
with InvertedIndexWriter('test', directory='tmp/') as index:
    # Some code here
```

If you are unfamiliar with context managers in python, we would recommend checking out these tutorials ([1](https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/), [2](http://arnavk.com/posts/python-context-managers/)), although it isn't necessary to be able to understand all of it.

## Indexing

> BSBI (i) segments the collection into parts of equal size, (ii) sorts the termID-docID pairs of each part in memory, (iii) stores intermediate sorted results on disk, and (iv) merges all intermediate results into the final index. 

You should treat each sub-directory as a block, and only load one block in memory at a time when you build your index (note: you will be penalized if you construct the index by loading blocks of larger than one directory in memory at once). Note that we are abstracting away from the operating systems meaning of blocks. You can assume that each block is small enough to be stored in memory.

In this section, we introduce the class `BSBIIndex` which we will progressively build. The `index` function provides the skeleton for BSBI and your job is to implement `parse_block`, `invert_write` and `merge` functions in subsequent sections.

In [15]:

# Do not make any changes here, they will be overwritten while grading
class BSBIIndex:
    """ 
    Attributes
    ----------
    term_id_map(IdMap): For mapping terms to termIDs
    doc_id_map(IdMap): For mapping relative paths of documents (eg 
        0/3dradiology.stanford.edu_) to docIDs
    data_dir(str): Path to data
    output_dir(str): Path to output index files
    index_name(str): Name assigned to index
    postings_encoding: Encoding used for storing the postings.
        The default (None) implies UncompressedPostings
    """
    def __init__(self, data_dir, output_dir, index_name = "BSBI", 
                 postings_encoding = None):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = index_name
        self.postings_encoding = postings_encoding

        # Stores names of intermediate indices
        self.intermediate_indices = []
        
    def save(self):
        """Dumps doc_id_map and term_id_map into output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pkl.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pkl.dump(self.doc_id_map, f)
    
    def load(self):
        """Loads doc_id_map and term_id_map from output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
            self.term_id_map = pkl.load(f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
            self.doc_id_map = pkl.load(f)
            
    def index(self):
        """Base indexing code
        
        This function loops through the data directories, 
        calls parse_block to parse the documents
        calls invert_write, which inverts each block and writes to a new index
        then saves the id maps and calls merge on the intermediate indices
        """
        for block_dir_relative in sorted(next(os.walk(self.data_dir))[1]):
            td_pairs = self.parse_block(block_dir_relative)
            index_id = 'index_'+block_dir_relative
            self.intermediate_indices.append(index_id)
            with InvertedIndexWriter(index_id, directory=self.output_dir, 
                                     postings_encoding=
                                     self.postings_encoding) as index:
                self.invert_write(td_pairs, index)
                td_pairs = None
        self.save()
        with InvertedIndexWriter(self.index_name, directory=self.output_dir, 
                                 postings_encoding=
                                 self.postings_encoding) as merged_index:
            with contextlib.ExitStack() as stack:
                indices = [stack.enter_context(
                    InvertedIndexIterator(index_id, 
                                          directory=self.output_dir, 
                                          postings_encoding=
                                          self.postings_encoding)) 
                 for index_id in self.intermediate_indices]
                self.merge(indices, merged_index)

### Parsing

> The function `parse_block`  parses documents into termID-docID pairs and accumulates the pairs in memory until a block of a fixed size is full. We choose the block size to fit comfortably into memory to permit a fast in-memory sort. 

You should treat each sub-directory as a block; `parse_block` expects the path to the sub-directory which is the block. You should assume that the individual file names are unique within each sub-directory, but not necessarily the case across sub- directories (i.e., the full path of the files are unique).

_NB - We are inheriting `BSBIIndex` from `BSBIIndex`, which is just a simple way of adding a new method to an existing class. Please do not be confused, it is simply used as a pedagogical tool to split a class definition within a jupyter notebook and there is no other magic happening here._

In [16]:
%%tee submission/parse_block.py
class BSBIIndex(BSBIIndex):            
    def parse_block(self, block_dir_relative):
        """Parses a tokenized text file into termID-docID pairs
        
        Parameters
        ----------
        block_dir_relative : str
            Relative Path to the directory that contains the files for the block
        
        Returns
        -------
        List[Tuple[Int, Int]]
            Returns all the td_pairs extracted from the block
        
        Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
        These persist across calls to parse_block
        """
        ### Begin your code
        pairs = []
        
        filenames = next(os.walk(os.path.join(self.data_dir,block_dir_relative)))[2]
        path_prefix = os.path.join(self.data_dir,block_dir_relative)
        for filename in filenames:
            full_path = os.path.join(path_prefix,filename)
            
            ### map doc to id
            docID = self.doc_id_map[full_path]
            
            with open(full_path,'r') as f:
                lines = f.readlines()
                for line in lines:
                    line = re.sub(re.compile(r"\W+"),' ',line)
                    terms = line.split()
                    for term in terms:
                        
                        ### map term to id
                        termID = self.term_id_map[term]
                        
                        pairs.append(tuple((termID,docID)))
        return pairs
    
        ### End your code

Overwriting submission/parse_block.py


See if the function works as expected on the toy data.

In [17]:
with open('toy-data/0/fine.txt', 'r') as f:
    print(f.read())
with open('toy-data/0/hello.txt', 'r') as f:
    print(f.read())

i'm fine , thank you

hi hi
how are you ?



In [18]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
BSBI_instance.parse_block('0')
BSBI_instance.parse_block('1')
BSBI_instance.parse_block('2')

[(0, 4),
 (1, 4),
 (2, 4),
 (3, 4),
 (4, 4),
 (5, 5),
 (5, 5),
 (6, 5),
 (7, 5),
 (4, 5)]

Does the `parse_block` method work as expected on a block of the toy data? Write some tests to make sure that is indeed the case (for example, you should make sure that a given word gets the same id each time it appears).

In [19]:
### Begin your code
print(BSBI_instance.doc_id_map.str_to_id)
print(BSBI_instance.term_id_map.str_to_id)

### End your code

{'toy-data\\0\\fine.txt': 0, 'toy-data\\0\\hello.txt': 1, 'toy-data\\1\\bye.txt': 2, 'toy-data\\1\\byebye.txt': 3, 'toy-data\\2\\fine.txt': 4, 'toy-data\\2\\hello.txt': 5}
{'i': 0, 'm': 1, 'fine': 2, 'thank': 3, 'you': 4, 'hi': 5, 'how': 6, 'are': 7, 'good': 8, 'to': 9, 'see': 10, 'bye': 11, 'later': 12}


### Inversion

> The block is then inverted and written to disk. Inversion involves two steps. First, we sort the termID-docID pairs. Next, we collect all termID-docID pairs with the same termID into a postings list, where a posting is simply a docID. The result, an inverted index for the block we have just read, is then written to disk.

In this section we add the function `invert_write` which does exactly this. 

However, we first need to implement the class `InvertedIndexWriter`. This class provides an append function, just like a list, however the postings_list isn't stored in memory but is directly written to disk.

In [20]:
%%tee submission/inverted_index_writer.py
class InvertedIndexWriter(InvertedIndex):
    """"""
    def __enter__(self):
        self.index_file = open(self.index_file_path, 'wb+')              
        return self

    def append(self, term, postings_list):
        """Appends the term and postings_list to end of the index file.
        
        This function does three things, 
        1. Encodes the postings_list using self.postings_encoding
        2. Stores metadata in the form of self.terms and self.postings_dict
           Note that self.postings_dict maps termID to a 3 tuple of 
           (start_position_in_index_file, 
           number_of_postings_in_list, 
           length_in_bytes_of_postings_list)
        3. Appends the bytestream to the index file on disk

        Hint: You might find it helpful to read the Python I/O docs
        (https://docs.python.org/3/tutorial/inputoutput.html) for
        information about appending to the end of a file.
        
        Parameters
        ----------
        term:
            term or termID is the unique identifier for the term
        postings_list: List[Int]
            List of docIDs where the term appears
        """
        ### Begin your code   
        ### set self.terms
        self.terms.append(term)
        
        length = len(postings_list)
        postings_list_encoded = self.postings_encoding.encode(postings_list)
        size = len(self.postings_encoding.encode(postings_list))
        
        ### set self.postings_dict
        self.index_file.seek(0,2)
        start_position = self.index_file.tell()
        self.postings_dict[term] = (start_position,length,size)        
        
        ### write self.index_file from the end of the file
        self.index_file.seek(0,2)
        self.index_file.write(postings_list_encoded)
        

        ### End your code

Overwriting submission/inverted_index_writer.py


Even though we haven't written the classes which read the index, we can still test the implementation as follows. Make sure you pass the asserts before moving forward

In [21]:
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    index.index_file.seek(0)
    assert index.terms == [1,2], "terms sequence incorrect"
    #print(index.postings_dict)
    #print(len(UncompressedPostings.encode([2,3,4])))
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"

Now we implement `invert_write`, which takes in the td_pairs created with parse_block. We use `InvertedIndexWriter` to write them to disk.

In [22]:
%%tee submission/invert_write.py
class BSBIIndex(BSBIIndex):
    def invert_write(self, td_pairs, index):
        """Inverts td_pairs into postings_lists and writes them to the given index
        
        Parameters
        ----------
        td_pairs: List[Tuple[Int, Int]]
            List of termID-docID pairs
        index: InvertedIndexWriter
            Inverted index on disk corresponding to the block       
        """
        ### Begin your code
        postings_lists = {}
        
        ### construct postings_list
        for termID,docID in td_pairs:
            if termID in postings_lists:
                if docID not in postings_lists[termID]:
                    postings_lists[termID].append(docID)
            else:
                postings_lists[termID] = [docID]
        
        
        ### for debug
        #print('postings_list:',postings_lists)
        
        for termID,postings_list in postings_lists.items():
            index.append(termID,sorted(postings_list))

        ### End your code

Overwriting submission/invert_write.py


To test this on the toy data, we can read a block and see what the inverted index contains. Write some tests like you saw for the `InvertedIndexWriter`.

In [23]:
### Begin your code
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
for block_dir_relative in ['0','1','2']:
    td_pairs = BSBI_instance.parse_block(block_dir_relative)
    print(td_pairs)
    index_id = 'index_'+block_dir_relative
    with InvertedIndexWriter(index_id, directory='tmp/') as index:
        BSBI_instance.invert_write(td_pairs,index)
        index.index_file.seek(0,0)
        print(UncompressedPostings.decode(index.index_file.read()))
        print(index.postings_dict)
        print(index.terms)


### End your code

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 1), (5, 1), (6, 1), (7, 1), (4, 1)]
[0, 0, 0, 0, 0, 1, 1, 1, 1]
{0: (0, 1, 4), 1: (4, 1, 4), 2: (8, 1, 4), 3: (12, 1, 4), 4: (16, 2, 8), 5: (24, 1, 4), 6: (28, 1, 4), 7: (32, 1, 4)}
[0, 1, 2, 3, 4, 5, 6, 7]
[(8, 2), (9, 2), (10, 2), (4, 2), (11, 2), (10, 2), (4, 2), (12, 2), (11, 3), (11, 3), (11, 3)]
[2, 2, 2, 2, 2, 3, 2]
{8: (0, 1, 4), 9: (4, 1, 4), 10: (8, 1, 4), 4: (12, 1, 4), 11: (16, 2, 8), 12: (24, 1, 4)}
[8, 9, 10, 4, 11, 12]
[(0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 5), (5, 5), (6, 5), (7, 5), (4, 5)]
[4, 4, 4, 4, 4, 5, 5, 5, 5]
{0: (0, 1, 4), 1: (4, 1, 4), 2: (8, 1, 4), 3: (12, 1, 4), 4: (16, 2, 8), 5: (24, 1, 4), 6: (28, 1, 4), 7: (32, 1, 4)}
[0, 1, 2, 3, 4, 5, 6, 7]


### Merging

> The algorithm simultaneously merges the ten blocks into one large merged index. To do the merging, we open all block files simultaneously, and maintain small read buffers for the ten blocks we are reading and a write buffer for the final merged index we are writing. 

The iterator paradigm in Python lends itself naturally to our need of maintaining a small read buffer. We can iterate through the file while reading just one postings list at a time from the disk. We subclass `InvertedIndex` into `InvertedIndexIterator` which does the iteration bit. 

In [24]:
%%tee submission/inverted_index_iterator.py
class InvertedIndexIterator(InvertedIndex):
    """"""
    def __enter__(self):
        """Adds an initialization_hook to the __enter__ function of super class
        """
        super().__enter__()
        self._initialization_hook()
        return self

    def _initialization_hook(self):
        """Use this function to initialize the iterator
        """
        ### Begin your code
        self.already_got = 0

        ### End your code

    def __iter__(self): 
        return self
    
    def __next__(self):
        """Returns the next (term, postings_list) pair in the index.
        
        Note: This function should only read a small amount of data from the 
        index file. In particular, you should not try to maintain the full 
        index file in memory.
        """
        ### Begin your code
        ### for debug
        #print('call __next__')
        
        if self.already_got >= len(self.terms):
            raise StopIteration
        
        term = self.terms[self.already_got]
        
        start_position,length,size = self.postings_dict[term]
        self.index_file.seek(start_position,0)
        encoded_data = self.index_file.read(size)
        postings_list = self.postings_encoding.decode(encoded_data)
        
        self.already_got += 1
        return (term,postings_list)
        

        ### End your code

    def delete_from_disk(self):
        """Marks the index for deletion upon exit. Useful for temporary indices
        """
        self.delete_upon_exit = True

    def __exit__(self, exception_type, exception_value, traceback):
        """Delete the index file upon exiting the context along with the
        functions of the super class __exit__ function"""
        self.index_file.close()
        if hasattr(self, 'delete_upon_exit') and self.delete_upon_exit:
            os.remove(self.index_file_path)
            os.remove(self.metadata_file_path)
        else:
            with open(self.metadata_file_path, 'wb') as f:
                pkl.dump([self.postings_dict, self.terms], f)

Overwriting submission/inverted_index_iterator.py


Let's test this by first creating an index with the `InvertedIndexWriter` and then iterating through it. Write a few small test cases to see if it works. At the very least, you should write a test which manually constructs a small inverted index, writes them to disk with an `InvertedIndexWriter`, then uses an `InvertedIndexIterator` to iterate over the inverted index.

In [25]:
### Begin your code
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
for block_dir_relative in ['0','1','2']:
    td_pairs = BSBI_instance.parse_block(block_dir_relative)
    index_id = 'index_'+block_dir_relative
    with InvertedIndexWriter(index_id, directory='tmp/') as index:
        BSBI_instance.invert_write(td_pairs,index)
        print(index.postings_dict)
    with InvertedIndexIterator(index_id, directory='tmp/') as iterindex:
        #iterindex.delete_from_disk()
        for i in iterindex:
            print(i)
        
    

### End your code

{0: (0, 1, 4), 1: (4, 1, 4), 2: (8, 1, 4), 3: (12, 1, 4), 4: (16, 2, 8), 5: (24, 1, 4), 6: (28, 1, 4), 7: (32, 1, 4)}
(0, [0])
(1, [0])
(2, [0])
(3, [0])
(4, [0, 1])
(5, [1])
(6, [1])
(7, [1])
{8: (0, 1, 4), 9: (4, 1, 4), 10: (8, 1, 4), 4: (12, 1, 4), 11: (16, 2, 8), 12: (24, 1, 4)}
(8, [2])
(9, [2])
(10, [2])
(4, [2])
(11, [2, 3])
(12, [2])
{0: (0, 1, 4), 1: (4, 1, 4), 2: (8, 1, 4), 3: (12, 1, 4), 4: (16, 2, 8), 5: (24, 1, 4), 6: (28, 1, 4), 7: (32, 1, 4)}
(0, [4])
(1, [4])
(2, [4])
(3, [4])
(4, [4, 5])
(5, [5])
(6, [5])
(7, [5])


> During merging, in each iteration, we select the lowest termID that has not been processed yet using a priority queue or a similar data structure. All postings lists for this termID are read and merged, and the merged list is written back to disk. Each read buffer is refilled from its file when necessary.

We'll use the `InvertedIndexIterator` to do the reading part and `InvertedIndexWriter` to write the merged postings list. 

Note that we've been opening each inverted index using `with` statement, but now we need to do that programmatically. You can see that we've used `contextlib` ([documentation](https://docs.python.org/3/library/contextlib.html#contextlib.ExitStack)) for that in the provided `index` function of `BSBIIndex` base class. Your task is to **write the logic** for merging *opened* `InvertedIndexIterator` objects and writing one postings list at a time into into a single `InvertedIndexWriter` object. 

Since we know that the postings are sorted, we can appropriately merge them in sorted order in linear time. In fact `heapq` ([documentation](https://docs.python.org/3.0/library/heapq.html)) is a standard python module that provides an implementation of the heap queue algorithm. In particular it contains a `heapq.merge` utility function which merges multiple sorted inputs into a single sorted output and returns an iterator over the sorted values. Not only can this be handy with merging postings lists, but also with merging the inverted indices. 

To get you started on using the `heapq.merge` function, we've provided a sample usage of the function. The two lists contain animals/birds sorted by their average life span. We want to merge the two lists.

In [26]:
import heapq
animal_lifespans = [('Giraffe', 28), 
                   ('Rhinoceros', 40), 
                   ('Indian Elephant', 70), 
                   ('Golden Eagle', 80), 
                   ('Box turtle', 123)]

tree_lifespans = [('Gray Birch', 50), 
                  ('Black Willow', 70), 
                  ('Basswood', 100),
                  ('Bald Cypress', 600)]

lifespan_lists = [animal_lifespans, tree_lifespans]

for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
    print(merged_item)

('Giraffe', 28)
('Rhinoceros', 40)
('Gray Birch', 50)
('Indian Elephant', 70)
('Black Willow', 70)
('Golden Eagle', 80)
('Basswood', 100)
('Box turtle', 123)
('Bald Cypress', 600)


Notice the use of `*` to unpack `lifespan_lists` as arguments and `lambda` function to reprsent the key to perform the soring with. If you are unfamiliar you can check documentation ([unpacking lists](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists), [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions)) or tutorials ([unpacking lists](https://www.geeksforgeeks.org/packing-and-unpacking-arguments-in-python/), [lambda_expressions](https://www.afternerd.com/blog/python-lambdas/))

In [27]:
%%tee submission/merge.py

import heapq
class BSBIIndex(BSBIIndex):
    def merge(self, indices, merged_index):
        """Merges multiple inverted indices into a single index
        
        Parameters
        ----------
        indices: List[InvertedIndexIterator]
            A list of InvertedIndexIterator objects, each representing an
            iterable inverted index for a block
        merged_index: InvertedIndexWriter
            An instance of InvertedIndexWriter object into which each merged 
            postings list is written out one at a time
        """
        ### Begin your code
        term_postings_list = heapq.merge(*indices,key=lambda td:td[0])
        #print(list(term_postings_list))
        postings_lists = {}
        for termID,postings_list in term_postings_list:
            if termID not in postings_lists:
                postings_lists[termID] = postings_list
            else:
                postings_lists[termID].extend(postings_list)
        for termID,postings_list in postings_lists.items():
            merged_index.append(termID,sorted(set(postings_list)))
        ### End your code

Overwriting submission/merge.py


First make sure it works without errors on toy data

In [28]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'toy_output_dir', )
BSBI_instance.index()

Now lets index the full corpus

In [29]:
if os.path.isfile('output_dir/BSBI.dict'):
    print('already construct pa1-data postings list')
else:
    BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
    BSBI_instance.index()

already construct pa1-data postings list


If you're having errors with just the merging part of it, you can use the following code to debug it. 

In [30]:
"""
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(10)]
with InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:
    with contextlib.ExitStack() as stack:
        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]
        BSBI_instance.merge(indices, merged_index)
"""

"\nBSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )\nBSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(10)]\nwith InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:\n    with contextlib.ExitStack() as stack:\n        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]\n        BSBI_instance.merge(indices, merged_index)\n"

## Boolean conjunctive retrieval

We'll be implementing a function `retrieve` to BSBIIndex, which given a query string consisting of space separated tokens, returns a list of documents that contain each of the tokens in the query. However, we do not want to be iterating through the index nor loading the entire index to find the relevant terms. 

First we'll implement `InvertedIndexMapper` which subclasses `InvertedIndex` to add functionality for retrieving postings corresponding to a particular term by seeking to that location in the file.

In [31]:
%%tee submission/inverted_index_mapper.py
class InvertedIndexMapper(InvertedIndex):
    def __getitem__(self, key):
        return self._get_postings_list(key)
    
    def _get_postings_list(self, term):
        """Gets a postings list (of docIds) for `term`.
        
        This function should not iterate through the index file.
        I.e., it should only have to read the bytes from the index file
        corresponding to the postings list for the requested term.
        """
        ### Begin your code
        start_position,length,size = self.postings_dict[term]
        self.index_file.seek(start_position,0)
        encoded_data = self.index_file.read(size)
        postings_list = self.postings_encoding.decode(encoded_data)
        return postings_list

        ### End your code

Overwriting submission/inverted_index_mapper.py


Write a few tests to check your `_get_postings_list` implementation.

In [32]:
### Begin your code
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.load()
print(len(BSBI_instance.term_id_map.id_to_str))
print(BSBI_instance.term_id_map.id_to_str[:30])

with InvertedIndexMapper('BSBI',directory='output_dir') as mapindex:
    print(mapindex[18])


### End your code

326404
['3d', 'radiology', 'lab', 'stanford', 'university', 'school', 'of', 'medicine', 'and', 'quantitative', 'imaging', 'in', 'the', 'department', 'search', 'this', 'site', 'only', 'medical', 'sites', 'ways', 'to', 'give', 'find', 'a', 'person', 'alumni', 'lane', 'library', 'about']
[0, 2, 4, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 199, 245, 246, 247, 248, 249, 250, 285, 305, 306, 307, 308, 309, 310, 312, 313, 433, 435, 446, 462, 466, 468, 474, 486, 496, 519, 525, 555, 653, 714, 733, 950, 971, 1020, 1032, 1033, 1084, 1115, 1117, 1130, 1173, 1183, 1184, 1198, 1201, 1207, 1362, 1387, 1388, 1402, 1414, 1415, 1427, 1431, 1440, 1448, 1463, 1466, 1643, 1650, 1712, 1714, 1778, 1826, 1853, 1862, 1877, 1898, 1913, 1922, 1940, 1945, 1949, 1999, 2024, 2025, 2026, 2051, 2091, 2171, 2188, 2197, 2211, 2240, 2254, 2328, 2374, 2377, 2378, 2379, 2380, 2381, 2382, 2384, 2385, 2386, 2387, 2388, 2389, 2390, 2391, 2392, 2412, 2542, 2810, 2844, 2847, 3300, 3302, 3341, 3344, 3

Now that we can obtain postings lists corresponding to the query terms, we want to intersect them. However, à la merging, we can use the sorted-ness of these lists to intersect them very efficiently. Here we'll implement `sorted_intersect` which takes two sorted lists and return a sorted intersection of the elements.  

In [33]:
%%tee submission/sorted_intersect.py
def sorted_intersect(list1, list2):
    """Intersects two (ascending) sorted lists and returns the sorted result
    
    Parameters
    ----------
    list1: List[Comparable]
    list2: List[Comparable]
        Sorted lists to be intersected
        
    Returns
    -------
    List[Comparable]
        Sorted intersection        
    """
    ### Begin your code
    return sorted(list(set(list1) & set(list2)))

    ### End your code

Overwriting submission/sorted_intersect.py


Write a few simple test cases to make sure it works correctly

In [34]:
### Begin your code
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.load()

print(BSBI_instance.term_id_map.str_to_id['boolean'])
print(BSBI_instance.term_id_map.str_to_id['retrieval'])

with InvertedIndexMapper('BSBI',directory='output_dir') as mapindex:
    list1 = mapindex[44260]
    list2 = mapindex[7342]
print(sorted_intersect(list1,list2))
%timeit sorted_intersect(list1,list2)
### End your code

44260
7342
[10551, 10552, 30385, 43926, 43934, 43937, 43939, 43943, 43950, 43960, 43986, 44002, 44008, 44011, 44036, 44049, 44053, 44054, 44056, 69905, 83887]
17.9 µs ± 970 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Now we're ready to write the `retrieve` function using `InvertedIndexMapper` and `sorted_intersect`

In [35]:
%%tee submission/retrieve.py
class BSBIIndex(BSBIIndex):
    def retrieve(self, query):
        """Retrieves the documents corresponding to the conjunctive query
        
        Parameters
        ----------
        query: str
            Space separated list of query tokens
            
        Result
        ------
        List[str]
            Sorted list of documents which contains each of the query tokens. 
            Should be empty if no documents are found.
        
        Should NOT throw errors for terms not in corpus
        """
        if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
            self.load()

        ### Begin your code
        
        ### convert query to termID
        termIDs = []
        for term in query.split():
            termID = self.term_id_map.str_to_id[term]
            if termID not in termIDs:
                termIDs.append(termID)
        
        intersection = []
        with InvertedIndexMapper(index_name=self.index_name,directory=self.output_dir,postings_encoding=self.postings_encoding) as mapindex:
            intersection = mapindex[termIDs[0]]
            for termID in termIDs[1:]:
                intersection = sorted_intersect(intersection,mapindex[termID])
        ### convert docID to doc name
        intersection = list(map(lambda docID:self.doc_id_map.id_to_str[docID],intersection))
        return intersection
            

        ### End your code

Overwriting submission/retrieve.py


Let us test if our index works on the real corpus with a simple query

In [36]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.retrieve('boolean retrieval')

['pa1-data\\1\\cs276.stanford.edu_',
 'pa1-data\\1\\cs276a.stanford.edu_',
 'pa1-data\\3\\infolab.stanford.edu_TR_CS-TN-95-21.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 'pa1-data\\4\\nlp.stanford

You can also test if one of these pages truly contains the query terms by reading a file as follows

In [37]:
with open("pa1-data/1/cs276.stanford.edu_", 'r') as f:
    print(f.read())

cs276 information retrieval and web search cs 276 ling 286 information retrieval and web search spring 2011 pandu nayak and prabhakar raghavan lecture 3 units tu th 4 15 5 30 gates b03 tas sonali aggarwal ivan cui valentin spitkovsky and sandeep sripada staff e mail cs276 spr1011 staff lists stanford edu lectures are also available online and on television through scpd sitn course description basic and advanced techniques for text based information systems efficient text indexing boolean and vector space retrieval models evaluation and interface issues web search including crawling link based algorithms and web metadata text web clustering classification text mining syllabus additional information staff contact information piazzza we strongly recommend students to post questions to the course page on www piazzza com instead of sending emails this forum enables students to discuss the questions they encounter in lectures or assignments here is a quick introduction video email if you hav

Let's test on the dev queries to see if it works alright. 

In [38]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = BSBI_instance.retrieve(query)
        if not os.path.isdir('dev_out'):
            os.mkdir('dev_out')
        with open('dev_out/result.' + str(i),'w') as o:
            o.writelines('%s\n' % result for result in my_results)
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


If an assert is failing you can compare the difference in outputs as follows

In [39]:
#set(my_results) - set(reference_results)

In [40]:
#set(reference_results) - set(my_results)

Make sure you write your own queries test for all kinds of corner cases like terms which have never occurred in the corpus, or terms that are very frequent and might slow down the merge. 

# 构建压缩索引

In this section, you should build a compressed index using gap encoding with variable byte encoding for each gap.

Here are some things about python you'll find useful. 

In [41]:
# Remainder (modulo) operator %

print("10 % 2 = ", 10 % 2)
print("10 % 3 = ", 10 % 3)

# Integer division in Python 3 is done by using two slash signs

print("10 / 3 = ", 10 / 3)
print("10 // 3 = ", 10 // 3)

10 % 2 =  0
10 % 3 =  1
10 / 3 =  3.3333333333333335
10 // 3 =  3


Fill in the following `CompressedPostings` class which we shall use as a drop-in replacement for `UncompressedPostings`. You should refer to [Chapter 5](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf) of the textbook for understanding how gap encoding with variable byte encoding for each gap is done. 

In [42]:
%%tee submission/compressed_postings.py
class CompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code

    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` using gap encoding with variable byte 
        encoding for each gap
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
            (as produced by `array.tobytes` function)
        """
        ### Begin your code
        postings_list_gap = []
        postings_list_gap.append(postings_list[0])
        for x,y in zip(postings_list,postings_list[1:]):
            postings_list_gap.append(y-x)

        vb = []
        for gap in postings_list_gap:
            gap_bin_str = bin(gap)[2:]
            left = len(gap_bin_str) % 7
            gap_vb = []
            if left != 0:
                gap_vb.append(gap_bin_str[:left].zfill(8))
            gap_bin_str = gap_bin_str[left:]
            num_7bits = len(gap_bin_str) // 7
            for num in range(num_7bits):
                gap_vb.append(gap_bin_str[num*7:(num+1)*7].zfill(8))
            gap_vb[-1] = '1'+ gap_vb[-1][1:]
            vb.extend(gap_vb)

        result = b''
        for str_8bits in vb:
            int_8bits = int(str_8bits,2)
            bytes_8bits = struct.pack('B',int_8bits)
            result += bytes_8bits
        return result

        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docIds)
        """
        ### Begin your code
        docID_bin_str = ''
        postings_list_gap = []
        for byte in encoded_postings_list:
            bin_str = bin(byte)[2:]
            bin_str_padding = bin_str.zfill(8)
            if bin_str_padding[0] == '1':
                ### already read a docID
                docID_bin_str += bin_str_padding[1:]
                postings_list_gap.append(int(docID_bin_str,2))
                docID_bin_str = ''
            else:
                docID_bin_str += bin_str_padding[1:]
        postings_list = []
        postings_list.append(postings_list_gap[0])
        for gap in postings_list_gap[1:]:
            last = postings_list[-1]
            postings_list.append(last+gap)
        return postings_list

        ### End your code


Overwriting submission/compressed_postings.py


First, write some test cases for any helper methods that you might have implemented

In [43]:
### Begin your code
l = [25,68,525,569,789,792,795,813,888,1000,1001,*list(range(10000,90000))]
r_un = UncompressedPostings.encode(l)
print(len(r_un))
r = CompressedPostings.encode(l)
print(len(r))
CompressedPostings.decode(r) == UncompressedPostings.decode(r_un)

### End your code

320044
80014


True

We've provided a simple helper function for you to test whether an encoded postings list is decode correctly.

Write a few test cases to make sure the postings list compression and decompression is being done correctly

In [44]:
### Begin your code

### End your code

Now let us create a new output directory and use the `CompressedPostings` to create a compressed index

In [45]:
try: 
    os.mkdir('output_dir_compressed')
except FileExistsError:
    pass

In [46]:
if os.path.isfile('output_dir_compressed/BSBI.dict'):
    print('already construct pa1-data postings list')
else:
    BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
    BSBI_instance_compressed.index()

already construct pa1-data postings list


In [47]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.retrieve('boolean retrieval')

['pa1-data\\1\\cs276.stanford.edu_',
 'pa1-data\\1\\cs276a.stanford.edu_',
 'pa1-data\\3\\infolab.stanford.edu_TR_CS-TN-95-21.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 'pa1-data\\4\\nlp.stanford

As before, let us now test on the dev queries to see if it works alright

In [55]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', postings_encoding=CompressedPostings)
### test if compressed version and uncompressed version are same
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = BSBI_instance_compressed.retrieve(query)
        with open('dev_out/result.' + str(i)) as o:
            results = o.readlines()
        results = list(map(lambda s:s.strip(),results))
        assert my_results == results, "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


## Code submission

Now you are ready to submit the code part of your assignment. Refer to [submission instructions section](#Submission-instructions). 

# 其他压缩方法实现(选做)

Implement one more index compression method by filling in the `encode` and `decode` methods in `ECCompressedPostings`. 

Here are a few directions  
1. [Section 5.4](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf) in the textbook points to a few techniques (e.g., **gamma-encoding**) and related publications. 
2. You can also look into **Delta Encoding**, a compression technique very similar to Gamma Encoding, which can achieve an even better compression rate. 
3. Also, [ALGORITHM SIMPLE-9](https://github.com/manning/CompressionAlgorithms#simple-9) is an interesting encoding technique that you could also consider implementing.

You should store each postings list is stored in a multiple of bytes (rather than bits). The positions and lengths in index file are in multiples of bytes. 

In [ ]:
%%tee submission/ec_compressed_postings.py
class ECCompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code

    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` 
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
        """
        ### Begin your code

        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docId)
        """
        ### Begin your code

        ### End your code

**Write a description of your index compression method as well as a discussion of the space/speed trade-off of this additional compression method, in comparison to what we implemented in previous tasks.**

> Your answer here